<a href="https://colab.research.google.com/github/kethlyncampos/webscraper/blob/main/web_scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
!apt update
!apt install chromium-chromedriver
!pip install selenium

Hit:1 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:4 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:7 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:8 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:9 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:13 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64

In [10]:
!pip install iso8601

In [11]:
import sys
import requests
import pandas as pd
import random
from time import sleep
from datetime import datetime, timedelta
from bs4 import BeautifulSoup
from selenium import webdriver
import dateutil.parser
import pytz
import calendar
import iso8601


In [12]:
def req(url):
  options = webdriver.ChromeOptions()
  options.add_argument('--headless')
  options.add_argument('--no-sandbox')
  options.add_argument('--disable-dev-shm-usage')
  options.add_argument("user-agent=Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.125 Safari/537.36")
  browser = webdriver.Chrome(options=options)
  browser.get(url)
  lastHeight = browser.execute_script("return document.body.scrollHeight")
  while True:
      browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
      sleep(5)
      newHeight = browser.execute_script("return document.body.scrollHeight")
      if newHeight == lastHeight:
          break
      lastHeight = newHeight
  return BeautifulSoup(browser.page_source, 'html.parser')

In [13]:
page1 = req('https://br.yahoo.com/topics/bolsa-de-valores/')

In [14]:
links1 = page1.find_all('a', attrs={'class':'js-content-viewer','class':'rapidnofollow'})

In [15]:
lista_url = []
for link in links1:
  aux = link['href']
  if 'http' not in aux:
    aux = 'https://br.yahoo.com' + aux
  if 'noticias-do-dia-' not in aux:
    lista_url.append(aux)

In [ ]:
title, description, author, date, publication_time, collect_time = "", "", "", "", "", ""

for i in lista_url:
  print(i)
  temp = req(i)
  title = temp.find('h1', attrs={'data-test-locator':'headline'}).text
  author = temp.find('span', attrs={'class':'caas-author-byline-collapse'}).text
  date = temp.find('time')['datetime']
  break

In [17]:
page1

<html class="ltr yahoo-page height100" dir="ltr"><head>
<title>Yahoo is part of the Yahoo family of brands</title>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, initial-scale=1, maximum-scale=1, minimum-scale=1, user-scalable=no, viewport-fit=cover" name="viewport"/>
<link href="https://s.yimg.com/oa/build/css/site-ltr-d43e0c55.css" rel="stylesheet"/>
<link href="https://s.yimg.com/oa/build/images/favicons/yahoo.png" rel="icon" type="image/png"/>
<style nonce="">/*! Copyright 2017 Yahoo Holdings, Inc. All rights reserved. */
template{display:none}._yb_7b6bj{font-family:'Helvetica Neue',Helvetica,Tahoma,Geneva,Arial,sans-serif;font-weight:400;font-stretch:normal;direction:ltr;display:block;box-sizing:border-box;text-align:start;-webkit-font-smoothing:antialiased;z-index:1000;overflow-anchor:none}.ybar-ytheme-oneyahoo._yb_7b6bj,.ybar-ytheme-fuji2._yb_7b6bj{font-family:'YahooSans VF',YahooSans,'Helvetica Neue',Helvetica,Tahoma,Geneva,Ar

In [ ]:
date_now = datetime.now()
new_date = datetime.fromisoformat(date[:-1])
print(new_date)
new_date = pytz.timezone('America/Sao_Paulo').localize(new_date)
new_date_string = new_date.strftime('%Y-%m-%d %H:%M:%S')
print(new_date_string)

ValueError: ignored

In [ ]:
new_date = dateutil.parser.parse(date)
print(new_date)
new_date = new_date.astimezone(pytz.timezone('America/Sao_Paulo'))
new_date_string = new_date.strftime('%Y-%m-%d %H:%M:%S')
pytz.timezone('America/Sao_Paulo')

2021-11-26 12:30:00+00:00


<DstTzInfo 'America/Sao_Paulo' LMT-1 day, 20:54:00 STD>

In [ ]:
new_date = iso8601.parse_date(date)
new_date = new_date.replace(tzinfo=None)
print(new_date)
new_date = pytz.timezone('America/Sao_Paulo').localize(new_date)
aware

ParseError: ignored

In [ ]:
dataset = pd.DataFrame(,columns=['Título','Descrição','Autor','Data de Publicação',])